### Projeto de Predição da Evolução do Índice IFIX

O Índice IFIX é o resultado de uma carteira teórica de ativos de fundos imobiliários. Seu objetivo é ser o indicador do desempenho médio das cotações dos fundos imobiliários negociados nos mercados de bolsa e de balcão organizado da B3.

> Os dados do índice foram coletados do site da B3 e podem ser acessados [neste link](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-fundos-de-investimentos-imobiliarios-ifix-estatisticas-historicas.htm).

Para as variáveis preditoras, foram usados dados históricos dos indicadores abaixo: 

* Taxa SELIC: é gerada através da média ponderada das operações diárias titulos públicos, sendo as taxas praticadas balizadas pela SELIC meta.
* CDI: de forma semelhante, é a estabelecido pela média das taxas praticadas entre dos empréstimos diários interbancários. É também balizada pela SELIC meta.
* IGP-M: pesquisado entre os dias 21 do mês anterior e 20 do mês de referência; 
  * Composição: 60% Índice de Preços ao Produtor Amplo (IPA); 30%  Índice de Preços ao Consumidor (IPC); 10%  Índice Nacional de Custo da Construção (INCC). 
  * Abrangência setorial: Alimentação, Habitação, Vestuário, Saúde e Cuidados Pessoais, Educação, Leitura e Recreação, Transportes, Despesas Diversas e Comunicação, sob o ponto de vista do produtor e do consumidor;
Materiais e equipamentos, serviços e mão-de-obra
* INCC: pesquisado entre os dias 21 do mês anterior e 20 do mês de referência; 
  * Abrangência setorial: materiais, serviços e mão-de-obra mais relevantes para a construção civil.

> Dados da SELIC e CDI podem ser obitidos também no site da B3 [neste link](http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v05_template_informacoes_di.asp?str_Modulo=completo&int_Idioma=1&int_Titulo=6&int_NivelBD=2).

> Dados do IGP-M e INCC podem ser coletados nos lnks a seguir: [IGP-M](https://extra-ibre.fgv.br/IBRE/sitefgvdados/visualizaconsulta.aspx) e [INCC](https://extra-ibre.fgv.br/IBRE/sitefgvdados/visualizaconsulta.aspx). 




In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("Regressao IFIX").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/HisoricoCDI.csv,HisoricoCDI.csv,115462,1710016875000
dbfs:/FileStore/tables/HistoricoCDI.csv,HistoricoCDI.csv,115462,1710017252000
dbfs:/FileStore/tables/HistoricoIGPM-1.csv,HistoricoIGPM-1.csv,8414,1710686210000
dbfs:/FileStore/tables/HistoricoIGPM.csv,HistoricoIGPM.csv,6746,1710685001000
dbfs:/FileStore/tables/HistoricoIGP_M.csv,HistoricoIGP_M.csv,6746,1710684725000
dbfs:/FileStore/tables/HistoricoINCC-1.csv,HistoricoINCC-1.csv,6633,1710686210000
dbfs:/FileStore/tables/HistoricoINCC.csv,HistoricoINCC.csv,5305,1710684725000
dbfs:/FileStore/tables/HistoricoSELIC.csv,HistoricoSELIC.csv,43851,1710016875000
dbfs:/FileStore/tables/IfixEvolucaoMensal.csv,IfixEvolucaoMensal.csv,2347,1710016875000
dbfs:/FileStore/tables/adult_data.csv,adult_data.csv,5608318,1709852380000


In [0]:
 # diretórios dos arquivos a serem utilizados
historicoCDI="/FileStore/tables/HistoricoCDI.csv"
historicoSELIC="/FileStore/tables/HistoricoSELIC.csv"
historicoIFIX="/FileStore/tables/IfixEvolucaoMensal.csv"
historicoIGPM="/FileStore/tables/HistoricoIGPM-1.csv"
historicoINCC="/FileStore/tables/HistoricoINCC-1.csv" 

#### Dataset CDI

In [0]:
#lendo arquivos armazenados CSV com o esquema definido
cdi = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoCDI)

In [0]:
cdi.show(5)

+----------+-------------+-------+-----+------------+
|      Data|Nr. Operações| Volume|Média|Fator Diário|
+----------+-------------+-------+-----+------------+
|2013-01-02|           15|1892473| 6,92|  1,00026555|
|2013-01-03|           16|2007587| 6,92|  1,00026555|
|2013-01-04|           17|2138985| 6,93|  1,00026593|
|2013-01-07|           18|2208986| 6,92|  1,00026555|
|2013-01-08|           18|2246656| 6,91|  1,00026518|
+----------+-------------+-------+-----+------------+
only showing top 5 rows



In [0]:
cdi.printSchema()

root
 |-- Data: date (nullable = true)
 |-- Nr. Operações: integer (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Média: string (nullable = true)
 |-- Fator Diário: string (nullable = true)



In [0]:
import pyspark.sql.functions as F
cdi = cdi.select(F.col('Média').alias("CDI"), F.col('Data').alias("Data"))  # trocando nomes do cabaçalho e selecionando as colunas que preciso

In [0]:
from pyspark.sql.functions import dayofmonth, month, year, last_day
# Adicione uma nova coluna contendo o último dia de cada mês
cdiData = cdi.withColumn("ultimo_dia", last_day("Data"))
# Filtrando pelo último dia de cada mês
cdiData = cdiData.filter(dayofmonth("Data") == dayofmonth("ultimo_dia"))

cdiData.show(5)

+----+----------+----------+
| CDI|      Data|ultimo_dia|
+----+----------+----------+
|6,95|2013-01-31|2013-01-31|
|6,98|2013-02-28|2013-02-28|
|7,23|2013-04-30|2013-04-30|
|7,72|2013-05-31|2013-05-31|
|8,22|2013-07-31|2013-07-31|
+----+----------+----------+
only showing top 5 rows



In [0]:
cdiData = cdiData.select('Data', "CDI")
cdiData.show(5)

+----------+----+
|      Data| CDI|
+----------+----+
|2013-01-31|6,95|
|2013-02-28|6,98|
|2013-04-30|7,23|
|2013-05-31|7,72|
|2013-07-31|8,22|
+----------+----+
only showing top 5 rows



In [0]:
cdiFinal = cdiData.withColumn("CDI", F.regexp_replace(F.col("CDI"), "[,]", "."))  #troca o valor de "," para "." 

In [0]:
cdiFinal.show(5)

+----------+----+
|      Data| CDI|
+----------+----+
|2013-01-31|6.95|
|2013-02-28|6.98|
|2013-04-30|7.23|
|2013-05-31|7.72|
|2013-07-31|8.22|
+----------+----+
only showing top 5 rows



In [0]:
cdiFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- CDI: string (nullable = true)



In [0]:
# Trocando CDI para tipo double
from pyspark.sql.functions import col
cdiConcat = cdiFinal.withColumn("CDI", col("CDI").cast("double"))

In [0]:
cdiConcat.printSchema()

root
 |-- Data: date (nullable = true)
 |-- CDI: double (nullable = true)



In [0]:
display(cdiConcat)

Data,CDI
2013-01-31,6.95
2013-02-28,6.98
2013-04-30,7.23
2013-05-31,7.72
2013-07-31,8.22
2013-09-30,8.71
2013-10-31,9.32
2013-12-31,9.77
2014-01-31,10.27
2014-02-28,10.59


Databricks visualization. Run in Databricks to view.

#### Dataset IFIX

In [0]:
# Importanto dataset do IFIX
ifix = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoIFIX)

In [0]:
ifix.show(5)

+---+----+--------+
|Mês| Ano|   Valor|
+---+----+--------+
|  1|2013|1.613,76|
|  2|2013|1.597,88|
|  3|2013|1.579,32|
|  4|2013|1.549,00|
|  5|2013|1.549,36|
+---+----+--------+
only showing top 5 rows



In [0]:
# Concatenando das colunas Mês e Ano e ajustando ao modelo yyyy-MM-dd
from pyspark.sql.functions import concat, lit, to_date
ifixData = ifix.withColumn("Data", to_date(concat(col("Ano"), lit("-"), col("Mês"), lit("-01"))))


In [0]:
ifixData.show(5)

+---+----+--------+----------+
|Mês| Ano|   Valor|      Data|
+---+----+--------+----------+
|  1|2013|1.613,76|2013-01-01|
|  2|2013|1.597,88|2013-02-01|
|  3|2013|1.579,32|2013-03-01|
|  4|2013|1.549,00|2013-04-01|
|  5|2013|1.549,36|2013-05-01|
+---+----+--------+----------+
only showing top 5 rows



In [0]:
# Alterando a coluna Valor para nome IFIX
ifixData = ifixData.select(F.col('Valor').alias("IFIX"), F.col('Data').alias("Data"))
ifixData.show(5)

+--------+----------+
|    IFIX|      Data|
+--------+----------+
|1.613,76|2013-01-01|
|1.597,88|2013-02-01|
|1.579,32|2013-03-01|
|1.549,00|2013-04-01|
|1.549,36|2013-05-01|
+--------+----------+
only showing top 5 rows



In [0]:
# Retirando o ponto da casa de milhares
from pyspark.sql.functions import regexp_replace
ifixFinal = ifixData.withColumn("IFIX", regexp_replace("IFIX", "\.", ""))
ifixFinal.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613,76|2013-01-01|
|1597,88|2013-02-01|
|1579,32|2013-03-01|
|1549,00|2013-04-01|
|1549,36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
# Trocando virgula por ponto
ifixFinal = ifixFinal.withColumn("IFIX", F.regexp_replace(F.col("IFIX"), "[,]", "."))
ifixFinal.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613.76|2013-01-01|
|1597.88|2013-02-01|
|1579.32|2013-03-01|
|1549.00|2013-04-01|
|1549.36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
ifixFinal.printSchema()

root
 |-- IFIX: string (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
ifixConcat = ifixFinal.withColumn("IFIX", col("IFIX").cast("Double")) # trocando valor string para Double

In [0]:
ifixConcat.printSchema()

root
 |-- IFIX: double (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
ifixConcat.show(5)

+-------+----------+
|   IFIX|      Data|
+-------+----------+
|1613.76|2013-01-01|
|1597.88|2013-02-01|
|1579.32|2013-03-01|
| 1549.0|2013-04-01|
|1549.36|2013-05-01|
+-------+----------+
only showing top 5 rows



In [0]:
display(ifixConcat)

IFIX,Data
1613.76,2013-01-01
1597.88,2013-02-01
1579.32,2013-03-01
1549.0,2013-04-01
1549.36,2013-05-01
1437.76,2013-06-01
1427.11,2013-07-01
1375.07,2013-08-01
1400.93,2013-09-01
1423.1,2013-10-01


Databricks visualization. Run in Databricks to view.

In [0]:
ifixConcat.describe().show()

+-------+-----------------+
|summary|             IFIX|
+-------+-----------------+
|  count|              135|
|   mean|2243.024074074075|
| stddev|647.2107060950273|
|    min|          1282.04|
|    max|          3382.28|
+-------+-----------------+



#### Dataset SELIC  

In [0]:
selic = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoSELIC)

In [0]:
selic.show(5)

+----------+----------+
|      Data|Taxa SELIC|
+----------+----------+
|2014-01-02|       9,9|
|2014-01-03|       9,9|
|2014-01-06|       9,9|
|2014-01-07|       9,9|
|2014-01-08|       9,9|
+----------+----------+
only showing top 5 rows



In [0]:
selic.printSchema()

root
 |-- Data: date (nullable = true)
 |-- Taxa SELIC: string (nullable = true)



In [0]:
# Renomeando a coluna Taxa SELIC para apenas SELIC
selic = selic.select(F.col('Taxa SELIC').alias("SELIC"), F.col('Data').alias("Data"))
selic.show(5)

+-----+----------+
|SELIC|      Data|
+-----+----------+
|  9,9|2014-01-02|
|  9,9|2014-01-03|
|  9,9|2014-01-06|
|  9,9|2014-01-07|
|  9,9|2014-01-08|
+-----+----------+
only showing top 5 rows



In [0]:
# Trocando virgula por ponto
selic = selic.withColumn("SELIC", F.regexp_replace(F.col("SELIC"), "[,]", ".")) # trocando ',' por '.'
selic.show(5)

+-----+----------+
|SELIC|      Data|
+-----+----------+
|  9.9|2014-01-02|
|  9.9|2014-01-03|
|  9.9|2014-01-06|
|  9.9|2014-01-07|
|  9.9|2014-01-08|
+-----+----------+
only showing top 5 rows



In [0]:
# Adicione uma nova coluna contendo o último dia de cada mês
selicData = selic.withColumn("ultimo_dia", last_day("Data"))
# Filtrando pelo último dia de cada mês
selicData = selicData.filter(dayofmonth("Data") == dayofmonth("ultimo_dia"))
selicData.show(5)

+-----+----------+----------+
|SELIC|      Data|ultimo_dia|
+-----+----------+----------+
| 10.4|2014-01-31|2014-01-31|
|10.65|2014-02-28|2014-02-28|
|10.65|2014-03-31|2014-03-31|
| 10.9|2014-04-30|2014-04-30|
| 10.9|2014-06-30|2014-06-30|
+-----+----------+----------+
only showing top 5 rows



In [0]:
# Selecionando as colunas neessárias
selicFinal = selicData.select('Data', 'SELIC')
selicFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- SELIC: string (nullable = true)



In [0]:
# trocando valor string para double
selicConcat = selicFinal.withColumn("SELIC", col("SELIC").cast("double")) 
display(selicConcat)

Data,SELIC
2014-01-31,10.4
2014-02-28,10.65
2014-03-31,10.65
2014-04-30,10.9
2014-06-30,10.9
2014-07-31,10.9
2014-09-30,10.9
2014-10-31,11.15
2014-12-31,11.65
2015-03-31,12.65


Databricks visualization. Run in Databricks to view.

In [0]:
selicConcat.printSchema()

root
 |-- Data: date (nullable = true)
 |-- SELIC: double (nullable = true)



#### Dataset IGP-M

In [0]:
igpm = spark.read.format('csv').options(inferSchema=True, header='true',delimiter=';').load(historicoIGPM)

In [0]:
display(igpm)

Data,IGP-M
01/06/1989,"3,09E-05"
01/07/1989,"4,19E-05"
01/08/1989,"5,74E-05"
01/09/1989,"8,03E-05"
01/10/1989,"1,13E-04"
01/11/1989,"1,59E-04"
01/12/1989,"2,34E-04"
01/01/1990,"3,77E-04"
01/02/1990,"6,84E-04"
01/03/1990,"1,26E-03"


In [0]:
igpm.printSchema()

root
 |-- Data: string (nullable = true)
 |-- IGP-M: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, to_date, concat_ws, lit, date_format
from pyspark.sql.types import DateType

igpmData = igpm.withColumn("Data", to_date(col("Data"), "dd/MM/yyyy"))

igpmData.show(5)

+----------+--------+
|      Data|   IGP-M|
+----------+--------+
|1989-06-01|3,09E-05|
|1989-07-01|4,19E-05|
|1989-08-01|5,74E-05|
|1989-09-01|8,03E-05|
|1989-10-01|1,13E-04|
+----------+--------+
only showing top 5 rows



In [0]:
# Filtrando valores a partir do ano de 2013
igpmData = igpmData.filter(year(col("Data")) >= 2013)
igpmData.show(5)

+----------+-------+
|      Data|  IGP-M|
+----------+-------+
|2013-01-01|511,977|
|2013-02-01|513,467|
|2013-03-01|514,526|
|2013-04-01|515,276|
|2013-05-01|515,299|
+----------+-------+
only showing top 5 rows



In [0]:
igpmData.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IGP-M: string (nullable = true)



In [0]:
from pyspark.sql.functions import regexp_replace
igpmValor = igpmData.withColumn("IGP-M", F.regexp_replace(F.col("IGP-M"), "[,]", "."))
igpmValor.show(5)
                                

+----------+-------+
|      Data|  IGP-M|
+----------+-------+
|2013-01-01|511.977|
|2013-02-01|513.467|
|2013-03-01|514.526|
|2013-04-01|515.276|
|2013-05-01|515.299|
+----------+-------+
only showing top 5 rows



In [0]:
igpmValor.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IGP-M: string (nullable = true)



In [0]:
igpmFinal = igpmValor.withColumn("IGP-M", col("IGP-M").cast("float"))
igpmFinal.show(5)

+----------+-------+
|      Data|  IGP-M|
+----------+-------+
|2013-01-01|511.977|
|2013-02-01|513.467|
|2013-03-01|514.526|
|2013-04-01|515.276|
|2013-05-01|515.299|
+----------+-------+
only showing top 5 rows



In [0]:
display(igpmFinal)

Data,IGP-M
2013-01-01,511.977
2013-02-01,513.467
2013-03-01,514.526
2013-04-01,515.276
2013-05-01,515.299
2013-06-01,519.153
2013-07-01,520.508
2013-08-01,521.27
2013-09-01,529.085
2013-10-01,533.621


Databricks visualization. Run in Databricks to view.

In [0]:
igpmFinal.describe().show()

+-------+------------------+
|summary|             IGP-M|
+-------+------------------+
|  count|               134|
|   mean| 783.5648721723414|
| stddev|228.13797633515642|
|    min|           511.977|
|    max|          1193.337|
+-------+------------------+



#### Dataset INCC

In [0]:
incc = spark.read.format('csv').options(inferSchema=True, header='true', delimiter=';').load(historicoINCC)

In [0]:
display(incc)

Data,INCC
01/07/1996,"149,224"
01/08/1996,"150,334"
01/09/1996,"150,564"
01/10/1996,"150,915"
01/11/1996,"151,345"
01/12/1996,"152,079"
01/01/1997,"153,107"
01/02/1997,"153,588"
01/03/1997,"154,228"
01/04/1997,"155,358"


In [0]:
incc.printSchema()

root
 |-- Data: string (nullable = true)
 |-- INCC: string (nullable = true)



In [0]:
inccData = incc.withColumn("Data", to_date(col("Data"), "dd/MM/yyyy"))
inccData.show(5)

+----------+-------+
|      Data|   INCC|
+----------+-------+
|1996-07-01|149,224|
|1996-08-01|150,334|
|1996-09-01|150,564|
|1996-10-01|150,915|
|1996-11-01|151,345|
+----------+-------+
only showing top 5 rows



In [0]:
inccValor = inccData.withColumn("INCC", F.regexp_replace(F.col("INCC"), "[,]", "."))
inccValor.show(5)

+----------+-------+
|      Data|   INCC|
+----------+-------+
|1996-07-01|149.224|
|1996-08-01|150.334|
|1996-09-01|150.564|
|1996-10-01|150.915|
|1996-11-01|151.345|
+----------+-------+
only showing top 5 rows



In [0]:
inccFinal = inccValor.withColumn("INCC", col("INCC").cast("float"))
inccFinal.show(5)

+----------+-------+
|      Data|   INCC|
+----------+-------+
|1996-07-01|149.224|
|1996-08-01|150.334|
|1996-09-01|150.564|
|1996-10-01|150.915|
|1996-11-01|151.345|
+----------+-------+
only showing top 5 rows



In [0]:
inccFinal.describe().show()

+-------+------------------+
|summary|              INCC|
+-------+------------------+
|  count|               332|
|   mean| 492.1975475908762|
| stddev|269.02511345000295|
|    min|           149.224|
|    max|          1090.871|
+-------+------------------+



In [0]:
display(inccFinal)

Data,INCC
1996-07-01,149.224
1996-08-01,150.334
1996-09-01,150.564
1996-10-01,150.915
1996-11-01,151.345
1996-12-01,152.079
1997-01-01,153.107
1997-02-01,153.588
1997-03-01,154.228
1997-04-01,155.358


Databricks visualization. Run in Databricks to view.

# Concatenando Datasets

#### Ajustando os datasets

In [0]:
# Extraindo mês e ano da coluna de data
ifixReg = ifixConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
ifixReg = ifixReg.select('IFIX', 'Mes_Ano')
ifixReg.show(5)

+-------+-------+
|   IFIX|Mes_Ano|
+-------+-------+
|1613.76| 1-2013|
|1597.88| 2-2013|
|1579.32| 3-2013|
| 1549.0| 4-2013|
|1549.36| 5-2013|
+-------+-------+
only showing top 5 rows



In [0]:
# Extraindo mês e ano da coluna de data
cdiReg = cdiConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
cdiReg = cdiReg.select('CDI', 'Mes_Ano')
cdiReg.show(5)

+----+-------+
| CDI|Mes_Ano|
+----+-------+
|6.95| 1-2013|
|6.98| 2-2013|
|7.23| 4-2013|
|7.72| 5-2013|
|8.22| 7-2013|
+----+-------+
only showing top 5 rows



In [0]:
# Extraindo mês e ano da coluna de data
selicReg = selicConcat.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
selicReg = selicReg.select('SELIC', 'Mes_Ano')
selicReg.show(5)

+-----+-------+
|SELIC|Mes_Ano|
+-----+-------+
| 10.4| 1-2014|
|10.65| 2-2014|
|10.65| 3-2014|
| 10.9| 4-2014|
| 10.9| 6-2014|
+-----+-------+
only showing top 5 rows



In [0]:
selicReg.printSchema()

root
 |-- SELIC: double (nullable = true)
 |-- Mes_Ano: string (nullable = true)



In [0]:
igpmReg = igpmFinal.withColumn("Mes_Ano", concat(month("Data"), lit("-"), year("Data")))
igpmReg = igpmReg.select('IGP-M', 'Mes_Ano')
igpmReg.show(5)

+-------+-------+
|  IGP-M|Mes_Ano|
+-------+-------+
|511.977| 1-2013|
|513.467| 2-2013|
|514.526| 3-2013|
|515.276| 4-2013|
|515.299| 5-2013|
+-------+-------+
only showing top 5 rows



In [0]:
igpmReg.printSchema()

root
 |-- IGP-M: float (nullable = true)
 |-- Mes_Ano: string (nullable = true)



In [0]:
inccReg = inccFinal.withColumn('Mes_Ano', concat(month('Data'), lit('-'), year('Data')))
inccReg = inccReg.select('INCC', 'Mes_Ano')
inccReg.show(5)

+-------+-------+
|   INCC|Mes_Ano|
+-------+-------+
|149.224| 7-1996|
|150.334| 8-1996|
|150.564| 9-1996|
|150.915|10-1996|
|151.345|11-1996|
+-------+-------+
only showing top 5 rows



In [0]:
inccReg.printSchema()

root
 |-- INCC: float (nullable = true)
 |-- Mes_Ano: string (nullable = true)



#### Dataset dos Indices

In [0]:
indices = ifixReg.join(cdiReg, "Mes_Ano", "inner").join(selicReg, "Mes_Ano", "inner").join(igpmReg, 'Mes_Ano', 'inner').join(inccReg, 'Mes_Ano', 'inner')
indices.show(5)

+-------+-------+-----+-----+-------+-------+
|Mes_Ano|   IFIX|  CDI|SELIC|  IGP-M|   INCC|
+-------+-------+-----+-----+-------+-------+
| 1-2014|1282.04|10.27| 10.4|540.959|569.745|
| 2-2014|1330.42|10.59|10.65|543.038|572.254|
| 3-2014|1336.07|10.55|10.65|552.087|573.505|
| 4-2014|1350.76| 10.8| 10.9| 556.42|577.322|
| 6-2014|1387.63| 10.8| 10.9|551.554| 592.51|
+-------+-------+-----+-----+-------+-------+
only showing top 5 rows



In [0]:
indices.printSchema()

root
 |-- Mes_Ano: string (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- IGP-M: float (nullable = true)
 |-- INCC: float (nullable = true)



In [0]:
indices_date = indices.withColumn("Data", to_date("Mes_Ano", "M-yyyy"))
indices_date.printSchema()


root
 |-- Mes_Ano: string (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- IGP-M: float (nullable = true)
 |-- INCC: float (nullable = true)
 |-- Data: date (nullable = true)



In [0]:
indicesFinal = indices_date.select('Data', 'IFIX', 'CDI', 'SELIC', 'INCC', 'IGP-M')
indicesFinal.show(5)

+----------+-------+-----+-----+-------+-------+
|      Data|   IFIX|  CDI|SELIC|   INCC|  IGP-M|
+----------+-------+-----+-----+-------+-------+
|2014-01-01|1282.04|10.27| 10.4|569.745|540.959|
|2014-02-01|1330.42|10.59|10.65|572.254|543.038|
|2014-03-01|1336.07|10.55|10.65|573.505|552.087|
|2014-04-01|1350.76| 10.8| 10.9|577.322| 556.42|
|2014-06-01|1387.63| 10.8| 10.9| 592.51|551.554|
+----------+-------+-----+-----+-------+-------+
only showing top 5 rows



In [0]:
indicesFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- INCC: float (nullable = true)
 |-- IGP-M: float (nullable = true)



In [0]:
indicesFinal.count()

Out[131]: 85

In [0]:
display(indicesFinal)

Data,IFIX,CDI,SELIC,INCC,IGP-M
2014-01-01,1282.04,10.27,10.4,569.745,540.959
2014-02-01,1330.42,10.59,10.65,572.254,543.038
2014-03-01,1336.07,10.55,10.65,573.505,552.087
2014-04-01,1350.76,10.8,10.9,577.322,556.42
2014-06-01,1387.63,10.8,10.9,592.51,551.554
2014-07-01,1409.42,10.8,10.9,597.251,548.202
2014-09-01,1421.44,10.81,10.9,599.333,547.839
2014-10-01,1402.46,11.06,11.15,600.549,549.396
2014-12-01,1336.72,11.57,11.65,603.895,558.213
2015-03-01,1347.63,12.6,12.65,613.374,569.536


In [0]:
indicesFinal.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- INCC: float (nullable = true)
 |-- IGP-M: float (nullable = true)



# Iniciando o Processo de Regressão

In [0]:
indicesFinal.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|summary|              IFIX|              CDI|            SELIC|             INCC|            IGP-M|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|  count|                85|               85|               85|               85|               85|
|   mean|2319.9676470588242|9.484117647058808|9.499999999999984| 799.028160903033|814.7069407743567|
| stddev| 630.6641043992679| 4.09802893797379|4.105418892843248|163.1581881050782|225.7878608758626|
|    min|           1282.04|              1.9|              1.9|          569.745|          540.959|
|    max|            3360.0|            14.14|            14.15|         1090.871|         1190.882|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+



#### Assembler incial contendo apenas as variáveis CDI e SELIC

In [0]:
#transformando os dados (linhas) em vetores
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler = VectorAssembler(inputCols=['CDI', 'SELIC'], outputCol='features')  #define o objeto para transformação
indicesRegression = assembler.transform(indicesFinal) #aplica a transformação
indicesRegression.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- INCC: float (nullable = true)
 |-- IGP-M: float (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
indicesRegression = indicesRegression.select(['features','IFIX'])
display(indicesRegression)

features,IFIX
"Map(vectorType -> dense, length -> 2, values -> List(10.27, 10.4))",1282.04
"Map(vectorType -> dense, length -> 2, values -> List(10.59, 10.65))",1330.42
"Map(vectorType -> dense, length -> 2, values -> List(10.55, 10.65))",1336.07
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1350.76
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1387.63
"Map(vectorType -> dense, length -> 2, values -> List(10.8, 10.9))",1409.42
"Map(vectorType -> dense, length -> 2, values -> List(10.81, 10.9))",1421.44
"Map(vectorType -> dense, length -> 2, values -> List(11.06, 11.15))",1402.46
"Map(vectorType -> dense, length -> 2, values -> List(11.57, 11.65))",1336.72
"Map(vectorType -> dense, length -> 2, values -> List(12.6, 12.65))",1347.63


In [0]:
train, test = indicesRegression.randomSplit([0.7, 0.3], seed=123)

Criando o Modelo de Regressão

In [0]:
train.show(5)

+-----------+-------+
|   features|   IFIX|
+-----------+-------+
|  [1.9,1.9]| 2782.1|
|  [1.9,1.9]|2794.88|
|  [1.9,1.9]|2870.15|
|[2.15,2.15]|2733.12|
|[2.15,2.15]|2806.28|
+-----------+-------+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression  #biblioteca que contém o modelo de regressão

lr = LinearRegression(maxIter=10, labelCol='IFIX') # define o objeto a ser utilizado para regressão
lrModel = lr.fit(train)

In [0]:
#Coeficientes angulares e lineares (a e b) da reta de regressão
print(f'Intercepto: {lrModel.intercept}\nCoeficiente Angular: {lrModel.coefficients.values}')

Intercepto: 2812.8810682940625
Coeficiente Angular: [ 14296.34076984 -14320.67830101]


In [0]:
#print das estatísticas do modelo
modelsummary = lrModel.summary

print("Variância Explicada:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Erro médio quadrático: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)


Variância Explicada: 227253.48827756057
R_2:  0.5567481728946275
Erro médio quadrático:  180926.54592828883
+------------------+
|         residuals|
+------------------+
|15.460240941359189|
| 28.24024094135939|
|103.51024094135937|
|-27.43537626450916|
| 45.72462373549115|
+------------------+
only showing top 5 rows



Realizando a Previsão Através do Modelo

In [0]:
modelsummary.predictions.show(5)

+-----------+-------+------------------+
|   features|   IFIX|        prediction|
+-----------+-------+------------------+
|  [1.9,1.9]| 2782.1|2766.6397590586407|
|  [1.9,1.9]|2794.88|2766.6397590586407|
|  [1.9,1.9]|2870.15|2766.6397590586407|
|[2.15,2.15]|2733.12| 2760.555376264509|
|[2.15,2.15]|2806.28| 2760.555376264509|
+-----------+-------+------------------+
only showing top 5 rows



In [0]:
display(modelsummary.predictions)

features,IFIX,prediction
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2782.1,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2794.88,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(1.9, 1.9))",2870.15,2766.6397590586407
"Map(vectorType -> dense, length -> 2, values -> List(2.15, 2.15))",2733.12,2760.555376264509
"Map(vectorType -> dense, length -> 2, values -> List(2.15, 2.15))",2806.28,2760.555376264509
"Map(vectorType -> dense, length -> 2, values -> List(2.65, 2.65))",2861.15,2748.3866106762384
"Map(vectorType -> dense, length -> 2, values -> List(3.4, 3.4))",2816.46,2730.1334622938325
"Map(vectorType -> dense, length -> 2, values -> List(3.65, 3.65))",2603.62,2724.0490794997045
"Map(vectorType -> dense, length -> 2, values -> List(4.15, 4.15))",2754.89,2711.880313911434
"Map(vectorType -> dense, length -> 2, values -> List(4.9, 4.9))",2791.98,2693.627165529028


In [0]:
# Grafico de linhas do IFIX + predição para comparação
predictionsDF = modelsummary.predictions
predictionsPlot = predictionsDF.join(ifixReg, "IFIX", "inner")
predictionsPlot = predictionsPlot.select('Mes_Ano', 'IFIX', 'prediction')
display(predictionsPlot)

Mes_Ano,IFIX,prediction
8-2020,2782.1,2766.6397590586407
9-2020,2794.88,2766.6397590586407
12-2020,2870.15,2766.6397590586407
7-2020,2733.12,2760.555376264509
6-2020,2806.28,2760.555376264509
4-2021,2861.15,2748.3866106762384
5-2021,2816.46,2730.1334622938325
4-2020,2603.62,2724.0490794997045
6-2021,2754.89,2711.880313911434
10-2019,2791.98,2693.627165529028


Databricks visualization. Run in Databricks to view.

#### Assembler adicionando as variáveis IGP-M e INCC

In [0]:
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler2 = VectorAssembler(inputCols=['CDI', 'SELIC' , 'IGP-M', 'INCC'], outputCol='features')  #define o objeto para transformação
indicesRegression = assembler2.transform(indicesFinal) #aplica a transformação
indicesRegression.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- INCC: float (nullable = true)
 |-- IGP-M: float (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
indicesRegression = indicesRegression.select(['features','IFIX'])
display(indicesRegression)

features,IFIX
"Map(vectorType -> dense, length -> 4, values -> List(10.27, 10.4, 540.958984375, 569.7449951171875))",1282.04
"Map(vectorType -> dense, length -> 4, values -> List(10.59, 10.65, 543.0380249023438, 572.2540283203125))",1330.42
"Map(vectorType -> dense, length -> 4, values -> List(10.55, 10.65, 552.0869750976562, 573.5050048828125))",1336.07
"Map(vectorType -> dense, length -> 4, values -> List(10.8, 10.9, 556.4199829101562, 577.322021484375))",1350.76
"Map(vectorType -> dense, length -> 4, values -> List(10.8, 10.9, 551.5540161132812, 592.510009765625))",1387.63
"Map(vectorType -> dense, length -> 4, values -> List(10.8, 10.9, 548.2020263671875, 597.2509765625))",1409.42
"Map(vectorType -> dense, length -> 4, values -> List(10.81, 10.9, 547.8389892578125, 599.3330078125))",1421.44
"Map(vectorType -> dense, length -> 4, values -> List(11.06, 11.15, 549.39599609375, 600.5490112304688))",1402.46
"Map(vectorType -> dense, length -> 4, values -> List(11.57, 11.65, 558.2130126953125, 603.89501953125))",1336.72
"Map(vectorType -> dense, length -> 4, values -> List(12.6, 12.65, 569.5360107421875, 613.3740234375))",1347.63


In [0]:
train, test = indicesRegression.randomSplit([0.7, 0.3], seed=123)

In [0]:
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler = VectorAssembler(inputCols=['CDI', 'SELIC'], outputCol='features')  #define o objeto para transformação
indicesRegression = assembler.transform(indicesFinal) #aplica a transformação
indicesRegression.printSchema()

root
 |-- Data: date (nullable = true)
 |-- IFIX: double (nullable = true)
 |-- CDI: double (nullable = true)
 |-- SELIC: double (nullable = true)
 |-- INCC: float (nullable = true)
 |-- IGP-M: float (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
train.show(5)

+--------------------+-------+
|            features|   IFIX|
+--------------------+-------+
|[1.9,1.9,832.3129...| 2782.1|
|[1.9,1.9,868.4420...|2794.88|
|[1.9,1.9,934.7579...|2870.15|
|[2.15,2.15,792.42...|2806.28|
|[2.15,2.15,810.08...|2733.12|
+--------------------+-------+
only showing top 5 rows



In [0]:
from pyspark.ml.regression import LinearRegression  #biblioteca que contém o modelo de regressão

lr = LinearRegression(maxIter=10, labelCol='IFIX') # define o objeto a ser utilizado para regressão
lrModel = lr.fit(train)

In [0]:
#Coeficientes angulares e lineares (a e b) da reta de regressão
print(f'Intercepto: {lrModel.intercept}\nCoeficiente Angular: {lrModel.coefficients.values}')

Intercepto: -199.3859719802086
Coeficiente Angular: [ 1.82857430e+03 -1.88466487e+03 -1.65588574e+00  5.53768299e+00]


In [0]:
#print das estatísticas do modelo
modelsummary = lrModel.summary

print("Variância Explicada:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Erro médio quadrático: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)


Variância Explicada: 395461.63281228987
R_2:  0.9715550907042869
Erro médio quadrático:  11578.211449745791
+-----------------+
|        residuals|
+-----------------+
|26.28653699805045|
|48.01157548721039|
|57.44927120809734|
|71.32704618709658|
|-9.25364912225723|
+-----------------+
only showing top 5 rows



In [0]:
modelsummary.predictions.show(5)

+--------------------+-------+------------------+
|            features|   IFIX|        prediction|
+--------------------+-------+------------------+
|[1.9,1.9,832.3129...| 2782.1|2755.8134630019495|
|[1.9,1.9,868.4420...|2794.88|2746.8684245127897|
|[1.9,1.9,934.7579...|2870.15|2812.7007287919027|
|[2.15,2.15,792.42...|2806.28|2734.9529538129036|
|[2.15,2.15,810.08...|2733.12| 2742.373649122257|
+--------------------+-------+------------------+
only showing top 5 rows



In [0]:
display(modelsummary.predictions)

features,IFIX,prediction
"Map(vectorType -> dense, length -> 4, values -> List(1.9, 1.9, 832.31298828125, 801.7769775390625))",2782.1,2755.8134630019495
"Map(vectorType -> dense, length -> 4, values -> List(1.9, 1.9, 868.4420166015625, 810.9650268554688))",2794.88,2746.8684245127897
"Map(vectorType -> dense, length -> 4, values -> List(1.9, 1.9, 934.7579956054688, 842.6829833984375))",2870.15,2812.7007287919027
"Map(vectorType -> dense, length -> 4, values -> List(2.15, 2.15, 792.4290161132812, 788.6160278320312))",2806.28,2734.9529538129036
"Map(vectorType -> dense, length -> 4, values -> List(2.15, 2.15, 810.0830078125, 795.2349853515625))",2733.12,2742.373649122257
"Map(vectorType -> dense, length -> 4, values -> List(2.65, 2.65, 1027.2110595703125, 885.093017578125))",2861.15,2852.3944149502286
"Map(vectorType -> dense, length -> 4, values -> List(3.4, 3.4, 1069.2889404296875, 901.031982421875))",2816.46,2828.9152582087345
"Map(vectorType -> dense, length -> 4, values -> List(3.65, 3.65, 778.1010131835938, 784.5230102539062))",2603.62,2651.876799301356
"Map(vectorType -> dense, length -> 4, values -> List(4.15, 4.15, 1075.7330322265625, 921.7620239257812))",2754.89,2890.9730484763695
"Map(vectorType -> dense, length -> 4, values -> List(4.9, 4.9, 741.3330078125, 773.2730102539062))",2791.98,2580.3482678287533


In [0]:
# Grafico de linhas do IFIX + predição para comparação
predictionsDF = modelsummary.predictions
predictionsPlot = predictionsDF.join(ifixReg, "IFIX", "inner")
predictionsPlot = predictionsPlot.select('Mes_Ano', 'IFIX', 'prediction')
display(predictionsPlot)

Mes_Ano,IFIX,prediction
8-2020,2782.1,2755.8134630019495
9-2020,2794.88,2746.8684245127897
12-2020,2870.15,2812.7007287919027
6-2020,2806.28,2734.9529538129036
7-2020,2733.12,2742.373649122257
4-2021,2861.15,2852.3944149502286
5-2021,2816.46,2828.9152582087345
4-2020,2603.62,2651.876799301356
6-2021,2754.89,2890.9730484763695
10-2019,2791.98,2580.3482678287533


Databricks visualization. Run in Databricks to view.

# Conclusão

Analisando as variáveis podemos concluir que adicionando as variáveis IGP-M e INCC ao conjunto, acaba melhorando o fit do modelo. Temos então, que o resultado de R² indica que aproximadamente 97% da variabilidade na variável dependente (IFIX) pode ser explicada pelas variáveis independentes (indices) incluídas no modelo.